In [2]:
import pandas as pd
import numpy as np
import re

import numpy as np
import datetime as dt
from datetime import datetime   

import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import squarify

%matplotlib inline

In [3]:
# Modelos
import sklearn 
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

In [4]:
sns.set()
plt.rc('xtick', labelsize=10)     
plt.rc('ytick', labelsize=10)
import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('clientes.csv', delimiter=';')
print(f"Tamaño del Dataset: {df.shape}")


FileNotFoundError: [Errno 2] No such file or directory: 'clientes.csv'

Importante aclarar que para la <font color='green'>reproducibilidad de los resultados</font> . Sobre todo para empresas de seguros que necesitan repetir el pricing para una cartera con un riesgo determinado, deben usar una semilla y debe ser la misma para dar los mismos resultados.

In [ ]:
import random
random.seed(100)
random_state = 100

## Contexto

Una entidad bancaria acaba de realizar una campaña de marketing, cuyo objetivo buscaba lograr que los clientes contactados depositaran sus ahorros a plazo fijo. Los resultados fueron <font color='red'>peor</font>  de lo esperado.
Esta campaña se enfocó en un 10% de la cartera total de clientes del banco.

El banco contrata nuestros servicios como científic@ de datos y nos comunica los siguintes objetivos:

* Repetir una o más campañas en el transcurso de los siguientes 18 meses. 
* Maximizar la tasa de conversión de las mismas y entender los factores que influyen en esta tasa.


Nuestra tarea consiste en ayudar al banco a cumplir sus objetivos.

**Comentario Inicial**

El problema es de tipo comercial en donde una campaña asegura la conversión o no del cliente. Para esto también es importante determinar la cantidad de días que pasan para llegar a la conversión. Entendiendo entonces que existe un número óptimo de campañas dentro de los 18 meses (a ejemplo: si un cliente convierte a 90 días del contacto de la campaña, sólo podré hacer 6 campañas en 18 meses)

*Pasos a seguir*:

1. Preprocesamiento
2. Exploracion de la data e interpretacion
3. Regresion logistica
4. Arbol D3



\begin{equation*}
Target = {Realizó.plazo.fijo}
\end{equation*}

In [ ]:
df.info()

### Metadata

#### Datos clientes

El banco nos provee la lista de los ~4.5K clientes ("clientes.csv") que fueron contactados durante esta última campaña y nos informa que por el momento NO nos proveerá de mas datos que estos.

1. `age`: edad del clietne
2. `job`: tipo de trabajo
3. `marital`: estado civil
4. `education`: nivel de educación
5. `default`: indica si el cliente registra algún default crediticio
5. `balance`: saldo en dólares de la caja de ahorro del cliente
6. `housing`: indica si el cliente posee un préstamo hipotecario
7. `loan`: indica si el cliente posee algún préstamo personal

#### Datos del último contacto de la campaña actual
9. `contact`: canal de contacto
10. `day`: día de último contacto
11. `month`: mes de último contacto
12. `duration`: duracion del último contacto
13. `campaign`: cantidad de contactos durante la campaña

#### Datos campaña previa
14. `pdays`: días transcurridos desde el último contacto por una campaña previa
15. `previous`: cantidad de contactos previous a esta campaña
16. `poutcome`: resultado de la campaña previa

#### Resultado campaña actual
17. `y` - Indica si el cliente realizó depósito a plazo fijo(binaria: 'yes','no')

In [ ]:
df.head().T

In [ ]:
df.previous.unique()

In [ ]:
df.y.unique()

Observo las medidas de estadística descritptiva de las variables numericas. La edad promedio es de 41 años, el balance promedio de cliente es 1422, el día promedio de contacto es el día 155, la duración promedio del ultimo contacto es de 259, la cantidad de contactos promedio es de 2.79.

In [ ]:
df.describe().round(2)

Será interesante estudiar cada variable por separado. Entendiendo las distrubciones a fin de entender que factores influyen en las campañas. 
* **Distribucion del target** `y`  determinar la conversión del cliente es nuestro target, por lo tanto complementarlo en el analisis de dato a las variables anteriores nos permitirá entender cómo se compone nuestra muestra.

* **Edad y rango etario** la edad es una variable importante a la hora de segmentar productos. Por lo tanto enfocaremos cómo punto de partida la distribución del target en función de  `age` .

* **Trabajo** El tipo de trabajo puede determinar la compra o no de productos financieros, en este caso entendemos al plazo fijo cómo un tipo de ahorro en un activo libre de riesgo.

* **Estacionalidad**: `days` y `month` los días de la semana y los meses determinaran la estacionalidad tal vez propia del producto (Si este caso fuese Argentina una hipotesis sería que los meses de aguinaldo la tasa de conversión podría ser mayor). 

* **Variables Económicas** `loan`, `balance`y `default`  seran también interesantes para segmentar. Será interesante hacer visualizaciones y analisis bivariados.

* Medios: `campaign` y  `contact`  Respecto al canal de campaña y las caracteristicas de contacto, establecer el comportamiento descriptivo de la data permite entender la operatividad de las llamadas y complementar luego a los modelos de predicción para elaborar una campaña con mayor probabilidad de conversión.

* Segmento target social: `marital`, `education`, no permitiran determinar la variables socioeconómicas de los clientes, sumado a las **data driven** de la campaña de marketing. 


### Heatmap: Correlaciones

In [ ]:
rcParams['figure.figsize'] = 8,6
sns.heatmap(df.corr(),cmap='coolwarm',annot=True)

Se observa una correlacion de 0.58 entre `pdays` y  `previous`, es decir que los días transcurridos entre el último contacto y la cantidad de contactos previos se relacionan positivamente. Ampliaremos esta relación, pero entendemos que en los posteriores modelos nos quedaremos sólo con una de la dos variables, tratando de evitar la multicolinealidad. Entendiendo que un modelo más simple para una cuestión interpretativa es mejor (*Occam's razor*)

## Preprocesamiento

### Missing Data
Chequeos si existen valores faltantes

In [ ]:
#Porcentaje de nulos por variable en df
(df.isnull().sum().sort_values()/len(df))*100

El dataset no posee faltantes. 

In [ ]:
df.month = pd.to_datetime(df.month, format='%b').dt.month

Convierto la columna mes al tipo datetime

## Distribución del target

In [ ]:
rcParams['figure.figsize'] = 6,6
paleta = ["#fc8d59","#3288bd"]

df['y'].value_counts()
sns.countplot(x='y', data=df, palette=paleta)
plt.xlabel("Dépositó el plazo fijo") 
plt.ylabel("Cantidad de Clientes")
plt.title("Distribución del Target")  
plt.show()



PF_NO = len(df[df['y']=='no'])
PF_YES = len(df[df['y']=='yes'])
no_plazofijo_porciento =( PF_NO / (PF_NO + PF_YES) * 100)
si_plazofijo_porciento= (PF_YES / (PF_NO + PF_YES) * 100)
print('Porcentajes de plazo fijo en la muestra:', si_plazofijo_porciento)
print('Porcentajes de NO plazo fijo en la muestra:', no_plazofijo_porciento)

Tenemos una distribución no equilibrada de las clases por lo cual tendremos que evaluar si realizamos algun método de sobresampleo o no en el futuro modelo.

Agrupo por target para ver la media según si hicieron el plazo fijo o no lo hicieron

In [ ]:
df.groupby('y').mean()

Se observa que los clientes que obtuvieron un plazo fijo tienen una edad media y un balance no es mucho mayor que los que no sacaron el plazo fijo. La diferencia en el valor de tendencia central se observa en la duración del ultimo contacto, la cual es mayor en los que convirtieron de los que no, y una cantidad mayor de días transcurridos desde el último contacto cómo también mayor cantidad de contactos previos.

Cómo primera intuición a validar con nuestros modelos observamos una relación positiva entre convertir (realizar el deposito) y las variables `duration`  y cantidad de contactos previos `previous`. Entendiendo que el promedio de los que convierten es 68 días entre llamada y llamada, cabe destacar que el promedio de  `pdays` días transcurridos es mayor en aquellos clientes que no convierten, teniendo que encontrar el punto óptimo entre cantidad de contactos en 180 días y el tiempo entre contacto.

In [ ]:
df1 = df.copy()

In [ ]:
# Paso a numerico el target para construir una correlacion pero viendo con que variable correlaciona
df1.y = df1.y.map({'no':0, 'yes':1}).astype('int32')

corr = df1.corr()
corr.style.background_gradient(cmap='PuBu')

In [ ]:
del df1

#### Duration
Teniendo en cuenta la variable Y, en la matriz de correlacion observamos la variable `duration`  es la que más se correlaciona cón el target. Habrá que realizar el analisi con la variable duracion transformada y sin, para ver si esta correlacion nos imposibilitan la capacidad de generalizar del modelo.  Un posible approach sea generar bins respecto a la duración separandola en cuartiles. Ahora bien hay que tener en cuenta dos perspectivas de la variable `duration` , por un lado puede darnos perspectiva de cómo deben ser las caracteristicas de las llamadas, esta data se utiliza en Speech Analytics. Por otro lado, hay que entender que puede ser útil cómo variable historica, pero en un set de datos nuevos (clientes aún no llamados) existe la probabilidad que no tenga aún esta variable. Estudiaremos esto en la parte de Feature Selection.

In [ ]:
paleta = ["#fc8d59","#3288bd"]
yes_df = df[df['y'] == 'yes']
no_df = df[df['y'] == 'no']

In [ ]:
#Por la reproducibilidad de los graficos prefiero siempre GGPLOT a estos códigos de seaborn y matplot


fig, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2, figsize = (13, 5))


sns.boxplot(y='duration', x='y', data=df, palette=paleta, ax = ax1)
ax1.set_xlabel('Llamadas', fontsize=10)
ax1.set_ylabel('Duración', fontsize=10)
ax1.set_title('Boxplot Duracion', fontsize=10)
ax1.tick_params(labelsize=10)


sns.distplot(yes_df['duration'],color='#3288bd', ax = ax2)
sns.distplot(no_df['duration'],color = '#fc8d59', ax = ax2) 
ax2.set_xlabel('Duracion del ultimo Contacto', fontsize=10)
ax2.set_ylabel('Ocurrencia', fontsize=10)
ax2.set_title('Frecuencia de datos de la variable Duracion', fontsize=10)
ax2.tick_params(labelsize=10)



plt.subplots_adjust(wspace=0.5)

plt.tight_layout() 

In [ ]:
print('1º Cuartil: ', yes_df['duration'].quantile(q = 0.25))
print('2º Cuartil: ', yes_df['duration'].quantile(q = 0.50))
print('3º Cuartil: ', yes_df['duration'].quantile(q = 0.75))
print('4º Cuartil: ', yes_df['duration'].quantile(q = 1.00))   
print('La duración arriba de: ', yes_df['duration'].quantile(q = 0.75) + 
                      1.5*(yes_df['duration'].quantile(q = 0.75) - yes_df['duration'].quantile(q = 0.25)), 'son outliers para clientes que depositaron el plazo fijo')

In [ ]:
print('1º Cuartil: ', no_df['duration'].quantile(q = 0.25))
print('2º Cuartil: ', no_df['duration'].quantile(q = 0.50))
print('3º Cuartil: ', no_df['duration'].quantile(q = 0.75))
print('4º Cuartil: ', no_df['duration'].quantile(q = 1.00))
print('La duración arriba de: ', no_df['duration'].quantile(q = 0.75) + 
                      1.5*(no_df['duration'].quantile(q = 0.75) - no_df['duration'].quantile(q = 0.25)), 'son outliers para los clientes que no depositaron el plazo fijo ')

## Edad y Rango etario

In [ ]:
print('Edad Max: ', df['age'].max())
print('Mediana de la edad: ', df['age'].median())
print('Promedio de la edad: ', df['age'].mean())
print('Edad Min: ', df['age'].min())

In [ ]:
rcParams['figure.figsize'] = 10,6
sns.countplot(x = 'age', data = df,color = "#607d8b")
plt.xlabel("Edad") 
plt.ylabel("Cantidad de Clientes")
plt.title("Distribución de la edad en la muestra")  



In [ ]:
fig, (ax1) = plt.subplots( figsize = (13, 5))


sns.boxplot(y='age', x='y', data=df, palette=paleta, ax = ax1)
ax1.set_xlabel('Conversión', fontsize=10)
ax1.set_ylabel('Edad', fontsize=10)
ax1.set_title('Boxplot Edad', fontsize=10)
ax1.tick_params(labelsize=10)



plt.subplots_adjust(wspace=0.5)

plt.tight_layout() 

Observamos que el gráfico de edad tiene una asimetria hacia la izquierda.

¿Qué pasa si cruzamos edad con duración? ¿Donde se concentran la mayor cantidad de valores en toda la muestra separando entonces conversión de no conversión?

In [ ]:
sns.jointplot(x='age', y='duration', data=yes_df[( yes_df['duration'] < 1497) & (yes_df['age'] < 60)], kind='kde', color='#3288bd')

In [ ]:
sns.jointplot(x='age', y='duration', data=no_df[(no_df['duration'] < 1497) & (no_df['age'] < 60)], kind='kde', color="#fc8d59")

Observamos que el centro de masa cambia en duración y no respecto a la edad.

In [ ]:
del no_df
del yes_df

In [ ]:
df["age"].describe()

#### Agrupacion por rango etareo
Agrupo las edades por decadas

In [ ]:
lst = [df]
for column in lst:
    column.loc[column["age"] < 30,  'grupo_edad'] = 20
    column.loc[(column["age"] >= 30) & (column["age"] <= 39), 'grupo_edad'] = 30
    column.loc[(column["age"] >= 40) & (column["age"] <= 49), 'grupo_edad'] = 40
    column.loc[(column["age"] >= 50) & (column["age"] <= 59), 'grupo_edad'] = 50
    column.loc[column["age"] >= 60, 'grupo_edad'] = 60

In [ ]:
total_por_edad = pd.crosstab(df['y'],df['grupo_edad'])
total_por_edad.loc['Total',:]= total_por_edad.sum(axis=0)
total_por_edad

In [ ]:
crosstab_edad = pd.crosstab(df['y'],df['grupo_edad']).apply(lambda x: x/x.sum() * 100)
crosstab_edad = crosstab_edad.transpose() 

In [ ]:
age = pd.DataFrame(df['grupo_edad'].value_counts())
age['% Decada'] = age['grupo_edad']*100/age['grupo_edad'].sum()
age['% Conversion'] = crosstab_edad['yes']
age.drop('grupo_edad',axis = 1,inplace = True)

age['age'] = [30,40,50,20,60]
age = age.sort_values('age',ascending = True)

In [ ]:
age

In [ ]:
rcParams['figure.figsize'] = 8,6
sns.set_style("darkgrid")
plot_age = age[['% Conversion','% Decada']].plot(kind = 'bar',color = ('#2ca25f','#b2e2e2'))
plt.xlabel('Edad del grupo')
plt.ylabel('Tasa de Conversión sobre el total')
plt.xticks(np.arange(5), ('<30', '30-39', '40-49', '50-59', '60+'),rotation = 'horizontal')
plt.title('Conversión vs Rango Etareo')
plt.show()

In [ ]:
del total_por_edad
del crosstab_edad
del age

Apuntar las llamadas a los extremos.

Cuando observamos la conversión por grupo etareo cambia la intuición al ver el total de las edades. Observamos que la conversión de los clientes mayores a 60 años es la más alta llegando a un 30% y a su vez aproximadamente un 15% de las conversiones son del extremo inferior, aquellos clientes con menos de 30 años.

Si dentro de la literatura financiera entendemos que un plazo fijo es un activo libre de riesgo (nunca en argentina), tendría sentido que los extremos etareos tengan aversión al riesgo, los jovenes porque pueden ahorrar para la educación y los mayores porque necesitan asegurar su retiro. 

Este grafico nos muestra que las llamadas del banco están orientadas al rango 30-40 y 40-49, con esta simple descripción de datos haría la primera recomendación de apuntar las llamadas a los extremos etareos. 

## Trabajos

In [ ]:
rcParams['figure.figsize'] = 14,6
trabajos = sns.countplot(x = 'job', data = df,
              order = df['job'].value_counts().index)
plt.xlabel("Trabajos")
plt.ylabel("Cantidad de Clientes")
plt.title("Distribucion de personas por trabajo") 
plt.show(trabajos)

In [ ]:
df1 = df.drop(df.loc[df["job"] == "unknown"].index) #Son solo 38 observaciones que para ver la distribucion del trabajo no me sirven
label_value = df1["job"].value_counts().to_dict()
# Creo los labels usando a list comprehesion
labels = ["{} tiene {} obs".format(class_, obs) for class_, obs in label_value.items()]
# Creo colores por cantidad de trabajos
colors = [plt.cm.Spectral(i/float(len(labels))) for i in range(len(labels))]
plt.figure(figsize = (18, 10))
squarify.plot(sizes = label_value.values(), label = labels,  color = colors, alpha = 0.8)
plt.title("Cantidad de trabajos por personas");

In [ ]:
cross_tab_trabajo = pd.crosstab(df['y'],df['job']).apply(lambda x: x/x.sum() * 100)
cross_tab_trabajo = cross_tab_trabajo.transpose()

In [ ]:
trabajo = cross_tab_trabajo['yes'].sort_values(ascending = True).plot(kind ='barh', color = "#2ca25f",
                                                                           figsize = (12,6))
                                                                               
plt.title('Tasa de conversion por trabajo')
plt.xlabel('Tasa de conversion')
plt.ylabel('Tipo de trabajo')

# Por estas funciones prefiero ggplot
for rec, label in zip(trabajo.patches,
                      cross_tab_trabajo['yes'].sort_values(ascending = True).round(1).astype(str)):
    trabajo.text(rec.get_width()+0.8, 
                  rec.get_y()+ rec.get_height()-0.5, 
                  label+'%', 
                  ha = 'center', 
                  va='bottom')

Consistentemente con la edad, las personas con mayor tasa de conversión son estudiantes o personas retiradas.

In [ ]:
del cross_tab_trabajo
del trabajo

## Estacionalidad

In [ ]:
fig, (ax2, ax3) = plt.subplots(nrows = 1, ncols = 2, figsize = (15,6))


sns.countplot(x='month', data=df, ax = ax2)
ax2.set_xlabel('Months', fontsize = 10)
ax2.set_ylabel('')
ax2.set_title('Meses del año')
ax2.tick_params()

sns.countplot(x='day', data=df, ax = ax3)
ax3.set_xlabel('Day of Week', fontsize = 10)
ax3.set_ylabel('')
ax3.set_title('Dia de la semana')
ax3.tick_params()

plt.subplots_adjust(wspace=0.25)

In [ ]:
fig, (ax2, ax3) = plt.subplots(nrows = 1, ncols = 2, figsize = (15,6))


sns.countplot(x="month", hue="y", palette=paleta, data=df, ax = ax2)
ax2.set_xlabel('Months', fontsize = 10)
ax2.set_ylabel('')
ax2.set_title('Meses del año')
ax2.tick_params()

sns.countplot(x='day', hue="y", palette=paleta, data=df, ax = ax3)
ax3.set_xlabel('Day of Week', fontsize = 10)
ax3.set_ylabel('')
ax3.set_title('Dia de la semana')
ax3.tick_params()

plt.subplots_adjust(wspace=0.25)

Aca grafiqué cantidad de llamadas con conversion y cantidad que no. Sería interesante graficar el % de conversión por mes y por día de la semana.

In [ ]:
mes_totales= pd.crosstab(df['y'],df['month'])
mes_totales_t  = mes_totales.T
mes_totales_t["total"] = mes_totales.T.sum(axis=1)
mes_totales_t


In [ ]:
crooss_tab_mes = pd.crosstab(df['y'],df['month']).apply(lambda x: x/x.sum() * 100)
crooss_tab_mes = crooss_tab_mes.transpose()

In [ ]:
mes_cant = pd.DataFrame(df['month'].value_counts())
mes_cant['% Llamadas totales'] = mes_cant['month']*100/mes_cant['month'].sum()
mes_cant['% Conversion'] = crooss_tab_mes['yes']
mes_cant.drop('month',axis = 1,inplace = True)

#me quedó el mes cómo columna con el cross tab y desordenado
mes_cant['mes'] = [5,7,8,6,11,4,2,1,10,9,3,12]
mes_cant = mes_cant.sort_values('mes',ascending = True)
#mes_cant


In [ ]:
rcParams['figure.figsize'] = 10,6

plot_month = mes_cant[['% Conversion','% Llamadas totales']].plot(kind ='line',color = ('#66c2a4','#3288bd'),                                                          marker = 'o')

plt.title('Conversion vs Porcentaje de llamadas')
plt.ylabel('Tasa de conversión y llamadas')
plt.xlabel('Meses')

ticks = np.arange(1,13,1)
#plt.xticks(ticks)

plt.show()

In [ ]:
mes_cant1 = mes_cant.copy()
mes_cant1.loc['Total',"% Llamadas totales"]=  mes_cant1["% Llamadas totales"].sum(axis=0)
mes_cant1

In [ ]:
print("La mayoría de las llamadas se realizaron en el mes de mayo, representando el %30  del total de llamadas")
mes_cant[mes_cant['% Llamadas totales'] == mes_cant['% Llamadas totales'].max()]



In [ ]:
print("El maximo de % Conversion  se encuentra en el mes 10 con un % 46 de conversion de llamadas pero sólo un % 1,76 de llamadas totales")
mes_cant[mes_cant['% Conversion'] == mes_cant['% Conversion'].max()]

La recomendación sería hacer más campañas en el mes 10

In [ ]:
#libero espacio del disco
del crooss_tab_mes
del mes_cant
del mes_cant1

In [ ]:
crooss_tab_dia = pd.crosstab(df['y'],df['day']).apply(lambda x: x/x.sum() * 100)
crooss_tab_dia = crooss_tab_dia.transpose()
dia_cant = pd.DataFrame(df['day'].value_counts())
dia_cant['% Llamadas totales por dia'] = dia_cant['day']*100/dia_cant['day'].sum()
dia_cant['% Conversion diario'] = crooss_tab_dia['yes']
dia_cant['dias'] = dia_cant.index


dia_cant = dia_cant.sort_values('dias',ascending = True)

rcParams['figure.figsize'] = 10,6

plot_dias = dia_cant[['% Conversion diario','% Llamadas totales por dia']].plot(kind ='line',color = ('#006d2c','#b2e2e2'),                                                          marker = 'o')

plt.title('Conversion vs Porcentaje de llamadas')
plt.ylabel('Tasa de conversión y llamadas')
plt.xlabel('Dias')

ticks = np.arange(1,31,1)
plt.xticks(ticks)

plt.show()


In [ ]:
print("El maximo de llamadas totales se encuentra concentrado en el día 20 y el cual es uno de los días con menor conversión.")
dia_cant[dia_cant['% Llamadas totales por dia'] == dia_cant['% Llamadas totales por dia'].max()]

In [ ]:
print("El maximo de % Conversion diario se encuentra en el 1er día del mes y el segundo pico se encuentra en el día 10. Podría tener sentido con los días en los que se cobra el sueldo")
dia_cant[dia_cant['% Conversion diario'] == dia_cant['% Conversion diario'].max()]

In [ ]:
#libero espacio
del dia_cant
del crooss_tab_dia

## Variables economicas

In [ ]:
# Colores = unique
color_labels = df['default'].unique()
rgb_values = sns.color_palette("Paired", 2)
#para usar la paleta paired por los labels 
color_map = dict(zip(color_labels, rgb_values))

In [ ]:

fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, ncols = 1, figsize = (10,10))
# Está en default?
sns.countplot(x = 'default', data = df, ax = ax3, order = ['no', 'yes'], palette=df['default'].map(color_map))
ax3.set_title('Default')
ax3.set_xlabel('')
ax3.set_ylabel('Cantidad')
ax3.tick_params()

# Tiene un prestamo hipotecario
sns.countplot(x = 'housing', data = df, ax = ax1, order = ['no',  'yes'], palette=df['housing'].map(color_map))
ax1.set_title('Housing')
ax1.set_xlabel('')
ax1.set_ylabel('Cantidad')
ax1.tick_params()

# Tiene un prestamo personal
sns.countplot(x = 'loan', data = df, ax = ax2, order = ['no', 'yes'], palette=df['loan'].map(color_map))
ax2.set_title('Loan')
ax2.set_xlabel('')
ax2.set_ylabel('Cantidad')
ax2.tick_params()

plt.subplots_adjust(wspace=0.8)

In [ ]:
print('Default:\n No tiene creditos en default'     , df[df['default'] == 'no']     ['default'].count(),
              '\n Tienen creditos en default:' , df[df['default'] == 'yes']    ['default'].count())

In [ ]:
print('Housing:\n No tienen prestamos hipotecarios:'     , df[df['housing'] == 'no']     ['housing'].count(),
              '\n tTnen prestamos hipotecarios:' , df[df['housing'] == 'yes']    ['housing'].count())

In [ ]:
print('Loan:\n No tienen prestamos personales:'     , df[df['loan'] == 'no']     ['loan'].count(),
              '\n Tienen prestamos personales:'    , df[df['loan'] == 'yes']    ['loan'].count())

## Balance

In [ ]:
fig, ax1 = plt.subplots(nrows = 1, ncols = 1, figsize = (5, 5))
sns.boxplot(y='balance', x='y', data=df, palette=paleta, ax = ax1)
ax1.set_xlabel('Balance', fontsize=10)
ax1.set_ylabel('Dinero', fontsize=10)
ax1.set_title('Boxplot Balance', fontsize=10)
ax1.tick_params(labelsize=10)
plt.subplots_adjust(wspace=0.5)
plt.tight_layout() 

In [ ]:
print('1º Cuartil: ', df['balance'].quantile(q = 0.25))
print('2º Cuartil: ', df['balance'].quantile(q = 0.50))
print('3º Cuartil: ', df['balance'].quantile(q = 0.75))
print('4º Cuartil: ', df['balance'].quantile(q = 1.00))
print('La duración arriba de: ',df['balance'].quantile(q = 0.75) + 
                      1.5*(df['balance'].quantile(q = 0.75) - df['balance'].quantile(q = 0.25)), 'son outliers')

No parece existir una política de llamadas. La dispersión del balance es muy grande. Sería interesante tener una idea para poder discretizar y separar entre clientes bajo, medio y alto. Pero cómo no sé en que tipo monetario está y no me puedo guiar por la distribución dejo este abordaje para posteriores estudios. 

### Estado civil y Educación

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2, figsize = (13, 5))
cross_tab_marital = pd.crosstab(df.marital, df.y)
cross_tab_marital.div(cross_tab_marital.sum(1).astype(float), axis=0).plot(kind='bar', stacked=False, color=['#fc8d59','#3288bd'], ax = ax1)
ax1.set_xlabel('Estado Civil', fontsize=10)
ax1.set_ylabel('% Conversion', fontsize=10)
ax1.set_title('Conversion por Estado Civil', fontsize=10)
ax1.tick_params(labelsize=10)


cross_tab_educacion = pd.crosstab(df.education, df.y)
cross_tab_educacion.div(cross_tab_educacion.sum(1).astype(float), axis=0).plot(kind='bar', stacked=False, color=['#fc8d59','#3288bd'], ax = ax2)

ax2.set_xlabel('Tipo de estudio', fontsize=10)
ax2.set_ylabel('% Conversion', fontsize=10)
ax2.set_title('Conversion  por tipo de estudio', fontsize=10)
ax2.tick_params(labelsize=10)


plt.subplots_adjust(wspace=0.5)

plt.tight_layout() 

In [ ]:
#libero espacio del disco
del cross_tab_marital
del cross_tab_educacion

## Campaña y contacto

In [ ]:
contacto = pd.crosstab(df.contact, df.y)
contacto.div(contacto.sum(1).astype(float), axis=0).plot(kind='bar', stacked=False, color=['#fc8d59','#3288bd'],grid=False, figsize=(10, 5))
plt.title("Conversion por modo de Contacto")
plt.show()

In [ ]:
del contacto

In [ ]:
df.campaign.unique()



In [ ]:
len(df.campaign.unique())

In [ ]:
campaña = pd.crosstab(df.campaign, df.y)
campaña.div(campaña.sum(1).astype(float), axis=0).plot(kind='bar', stacked=False, color=['#fc8d59','#3288bd'],grid=False, figsize=(15, 5))
plt.show()

In [ ]:
pd.crosstab(df['y'],df['campaign'])

In [ ]:
crooss_tab_camp = pd.crosstab(df['y'],df['campaign']).apply(lambda x: x/x.sum() * 100) 
crooss_tab_camp = crooss_tab_camp.transpose()
camp = pd.DataFrame(df['campaign'].value_counts())
camp['% Llamadas totales'] = camp['campaign']*100/camp['campaign'].sum()
camp['% Conversion'] = crooss_tab_camp['yes']
camp1 = camp
camp1.loc['Total',"% Llamadas totales"]=  camp["% Llamadas totales"].sum(axis=0) #suma el total de la columna que da 100% 
camp1['campañas'] = camp1.index
camp1 = camp1.sort_values('% Conversion',ascending = False)
camp1[camp1['% Conversion']>1]

In [ ]:
len(camp1[camp1['% Conversion']>1])

De las 32 campañas ejecutadas, sólo 14 superan el 1% de tasa de conversión. Recordemos que la tasa de conversión en este caso representa el ratio de clientes que realizaron el deposito sobre la cantidad que impacto esa campaña. Por ejemplo sólo se realizaron 3 llamadas de la campaña 24 y 1 de las llamadas fue una conversión. Habría que testear si esta tasa decae o se mantiene al aumentar la muestra.

En el caso de la campaña 1, representa el 38% de la muestra y tiene una tasa de conversión de 13%. Por lo cual podríamos establecerla cómo nuestra campaña estrella.

Cómo se muestra en el cuadro a continuación la campaña 11, 14, 15, 16, 18, 25, 19, 20, 28, 22, 32, 23,21,50,44,,29,30 y 31, deberían ser estudiadas para descartar, dado que latasa de conversión es menor al 1%

In [ ]:
camp1[camp1['% Conversion']<1].sort_values('% Llamadas totales',ascending = False)

In [ ]:
maximo = camp1[camp1['% Conversion']>1]
maximo['campañas'].unique()

In [ ]:
import plotly.express as px

fig = px.scatter(maximo, x="% Llamadas totales", y="% Conversion", color="campañas",
                 size='% Llamadas totales', hover_data=['campañas'],
                title="Campañas en función de tasa de Conversion y tasa de llamadas")
fig.show()


In [ ]:
del maximo
del campaña
del camp1
del camp

# Modelos

## Regresion Logistica

Recordemos que nuestro objetivo es:
* Repetir una o más campañas en el transcurso de los siguientes 18 meses.
* Maximizar la tasa de conversión de las mismas y entender los factores que influyen en esta tasa.

Por lo tanto nos interesa mantener todos las variables presentes, en caso de hacer un modelo de predicción de clientes deberíamos no elegir variables que no van a estar presentens en nuestra base de datos, cómo duration.

Ópte por un Regresión logística para tener interpretabilidad de los resultados, entendiendo a los coeficientes significativos cómo la probabilidad de ocurrencia.

### Selecion de variables por Random Forest

In [ ]:
data = df.copy()

Transformé a objetos las variables día, mes y campaña que por defecto las tomaba cómo entero.

In [ ]:
cat1 = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'day', 'month', 'campaign', 'poutcome',"grupo_edad"]

In [ ]:
for var in cat1:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(data[var], prefix=var)
    data1=data.join(cat_list)
    data=data1

In [ ]:
data_vars=data.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat1]
data_final=data[to_keep]
data_final.columns.values

In [ ]:
data_final['y'].unique()

In [ ]:
from sklearn import preprocessing
# Función para encodear
def encoder(df, cat):
    le = preprocessing.LabelEncoder()
    clases = []
    for i in cat:
        df[i]=le.fit_transform(df[i]) 
        clases.append(le.classes_)
    return df, clases

In [ ]:
data_final, target = encoder(data_final,"y")
target

In [ ]:
data_final = data_final.drop(['age'],axis=1)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
X=data_final.drop(['y'],axis=1)
y=data_final.y

model = RandomForestClassifier()
model.fit(X,y)

In [ ]:
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
columnas = list(feat_importances.nlargest(10).index)

In [ ]:
data_vars=data_final.columns.values.tolist()
to_keep=[i for i in data_vars if i in columnas]
x=data_final[to_keep]
x.columns.values

### Smote: Oversampling
El algoritmo SMOTE funciona en 4 sencillos pasos:
Elija un vector de entrada de clase minoritaria
Encuentre sus k vecinos más cercanos (k_neighbors se especifica como un argumento en la función SMOTE ())
Elija uno de estos vecinos y coloque un punto sintético en cualquier lugar de la línea que une el punto en consideración y su vecino elegido
Repita los pasos hasta que los datos estén equilibrados

In [ ]:
from imblearn.over_sampling import SMOTE

y = data_final.y

smote = SMOTE(random_state=100)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
columns = x_train.columns

smote_data_x,smote_data_y = smote.fit_sample(x_train, y_train)
smote_data_x = pd.DataFrame(data=smote_data_x, columns=columns)
smote_data_y = pd.DataFrame(data=smote_data_y, columns=['y'])



#### Balanceo de la clase

In [ ]:
print('Tamaño de la data antes de hacer oversampling: ', len(x_train))
print('Data oversampleada: ', len(smote_data_y))
print('Clientes que no depositaron: ', len(smote_data_y[smote_data_y['y'] == 0]))
print('Clientes que depositaron: ', len(smote_data_y[smote_data_y['y'] == 1]))

In [ ]:
import statsmodels.api as sm

In [ ]:
training_x = smote_data_x
training_y = smote_data_y['y']

In [ ]:
X_train_sm = sm.add_constant(training_x)
logm2 = sm.GLM(training_y,X_train_sm, family = sm.families.Binomial())
res = logm2.fit()
res.summary()

Coeficiente de balance es no representativo

\begin{align}
\dot{P(Deposito)} & = \beta_0+\beta_1 *balance+ \beta_2 *duration  + \beta_3 *pday+\beta_4 *previous
\beta_5 *maritalmarried+\beta_6 *month_10+\beta7 *campaign_1+\beta8 *campaign2+\beta9 *poutcomesucess +\beta910* grupo_edad_30
\end{align}

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(training_x, training_y,
                                                    test_size=0.3, random_state=100)

regressor = LogisticRegression()
regressor.fit(X_train, y_train)

In [ ]:
predictions = regressor.predict(X_test)
print('Accuracy on test set: ', regressor.score(X_test, y_test))

In [ ]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, predictions)

print('True Positive: ', confusion[0][0], 'False Negative: ', confusion[0][1])
print('False Positive: ', confusion[0][1], 'True Negative: ', confusion[1][1])

In [ ]:
# Precision, recall, F-measure ve support
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

In [ ]:
# ROC Curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

logit_roc_auc = roc_auc_score(y_test, predictions)
fpr, tpr, thresholds = roc_curve(y_test, regressor.predict_proba(X_test)[:, 1])

plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area=%0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.savefig('ROC-Curve')
plt.show()

La curva ROC es una herramienta común utilizada con clasificadores binarios. La línea de puntos representa un clasificador puramente aleatorio; un buen clasificador permanece lo más lejos posible de esa línea (hacia la esquina superior izquierda).

In [ ]:
y_train_pred = res.predict(X_train_sm)

In [ ]:
y_train_pred = y_train_pred.values.reshape(-1)
y_train_pred[:10]

In [ ]:
y_train_pred_final = pd.DataFrame({'Converted':training_y.values, 'Converted_prob':y_train_pred})
y_train_pred_final['ID'] = training_y.index
y_train_pred_final.head()

In [ ]:
y_train_pred_final['predicted'] = y_train_pred_final.Converted_prob.map(lambda x: 1 if x > 0.5 else 0)
from sklearn import metrics
# Matriz de confusión 
confusion = metrics.confusion_matrix(y_train_pred_final.Converted, y_train_pred_final.predicted )

In [ ]:
# Busco puntos de corte
numbers = [float(x)/10 for x in range(10)]
for i in numbers:
    y_train_pred_final[i]= y_train_pred_final.Converted_prob.map(lambda x: 1 if x > i else 0)
y_train_pred_final.head()

In [ ]:
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci'])
from sklearn.metrics import confusion_matrix



num = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in num:
    cm1 = metrics.confusion_matrix(y_train_pred_final.Converted, y_train_pred_final[i] )
    total1=cm1.sum()
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    
    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci]
print(cutoff_df)

In [ ]:
# El punto de corte se da en la interseccion de sensitivity  y specificity
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])
plt.show()

In [ ]:

y_train_pred_final['final_predicted'] = y_train_pred_final.Converted_prob.map( lambda x: 1 if x > 0.48 else 0)

y_train_pred_final.head()

In [ ]:
y_train_pred_final['Lead_Score'] = y_train_pred_final.Converted_prob.map( lambda x: round(x*100))

#calculo la accuracy del modelo ahora
metrics.accuracy_score(y_train_pred_final.Converted, y_train_pred_final.final_predicted)

confusion2 = metrics.confusion_matrix(y_train_pred_final.Converted, y_train_pred_final.final_predicted )
confusion2

TP = confusion2[1,1] # true positive 
TN = confusion2[0,0] # true negatives
FP = confusion2[0,1] # false positives
FN = confusion2[1,0] # false negatives

In [ ]:
#sensitivity
Recall = TP / float(TP+FN)
print(f"Recall: {Recall.round(2)}")
#specificity
specificity = TN / float(TN+FP)
print(f"Specificity: {specificity.round(2)}")
# false postive rate 
fprt = FP/ float(TN+FP)
print(f"False postive rate : {fprt.round(2)}")
# Positive predictive value 
ppv = TP / float(TP+FP)
print(f"Precision: {ppv.round(2)}")


In [ ]:
#Matriz de confusión
confusion = metrics.confusion_matrix(y_train_pred_final.Converted, y_train_pred_final.predicted )
confusion


In [ ]:
##### Chequeo Precision pero con la matriz de confusion
confusion[1,1]/(confusion[0,1]+confusion[1,1])

In [ ]:
##### Chequeo el valor de Recall en matriz
confusion[1,1]/(confusion[1,0]+confusion[1,1])